In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [ ]:
print('table creating')

In [2]:
target = pd.read_csv('target2',header=None,index_col=0)

In [3]:
train = feather.read_dataframe('best_from_recent_groupped_train')
test = feather.read_dataframe('best_from_recent_groupped_test')

In [4]:
test_cardid = test['card_id']

In [5]:
train = train[[c for c in train if c not in ['card_id', 'first_active_month','last_active_month']]]
test = test[[c for c in train if c not in ['card_id', 'first_active_month','last_active_month']]]

In [6]:
categorical_feats = [x for x in train if re.search('[th]_cat1_[^s]',x)]
categorical_feats += [x for x in train if (re.search('modeplus',x) or re.search('feature_',x))]
categorical_feats = list(set(categorical_feats))

In [7]:
cut = train.shape[0]
train = pd.concat([train,test])
train = pd.get_dummies(train,columns=categorical_feats)
test = train.iloc[cut:,:]
train = train.iloc[:cut,:]

In [8]:
#'grow_policy?'

In [ ]:
print('calculations starts')

In [ ]:
best_test = 10
best_x_test = np.nan
for x_test in range(1):
    params = {'max_depth': 4, 
              'eta': 0.00021, 
              'objective': 'reg:linear',
              'eval_metric': 'rmse',
              'nthreads':4,
              'subsample':0.9,
              'alpha':0.4,
              'tree_method':'gpu_hist',
              'grow_policy':'lossguide',
   #           'grow_policy':'depthwise',
              'max_leaves':50,
              'max_bin':87,
              'seed':77,
              'colsample_bytree':0.9,
              'min_child_weight':19,
              'max_delta_step':13,
              

             }

    folds = KFold(n_splits=5, shuffle=True, random_state=15)
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    feature_importance_df = pd.DataFrame()

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
        print("fold n°{}".format(fold_))

        dtrain = xgb.DMatrix(train.iloc[trn_idx],label=target.iloc[trn_idx])
        dval = xgb.DMatrix(train.iloc[val_idx],label=target.iloc[val_idx])
        dtest = xgb.DMatrix(test)


        clf = xgb.train(params=params,
              dtrain=dtrain,
              evals=[(dtrain, 'train'),(dval, 'eval')],
              num_boost_round=200000,
              early_stopping_rounds=5000,
              verbose_eval=5000)


        oof[val_idx] = clf.predict(dval)

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = clf.get_fscore().keys()
        fold_importance_df["importance"] = clf.get_fscore().values()
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        predictions += clf.predict(dtest) / folds.n_splits

    errr = mean_squared_error(oof, target)**0.5
    if errr < best_test:
        best_test = errr
        best_x_test = x_test
    
    print("CV score: {:<8.5f}\n".format(errr))
    print(x_test, '- x_test \n')

fold n°0
[0]	train-rmse:3.94362	eval-rmse:3.98899
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 5000 rounds.
[5000]	train-rmse:3.71958	eval-rmse:3.7657
[10000]	train-rmse:3.66506	eval-rmse:3.71571
[15000]	train-rmse:3.64132	eval-rmse:3.6981
[20000]	train-rmse:3.62512	eval-rmse:3.68924
[25000]	train-rmse:3.61163	eval-rmse:3.68359
[30000]	train-rmse:3.59965	eval-rmse:3.67981
[35000]	train-rmse:3.5886	eval-rmse:3.67702
[40000]	train-rmse:3.57844	eval-rmse:3.67503
[45000]	train-rmse:3.5688	eval-rmse:3.67363
[50000]	train-rmse:3.55953	eval-rmse:3.67258
[55000]	train-rmse:3.55042	eval-rmse:3.6717
[60000]	train-rmse:3.54149	eval-rmse:3.67105
[65000]	train-rmse:3.53276	eval-rmse:3.67049
[70000]	train-rmse:3.52415	eval-rmse:3.67
[75000]	train-rmse:3.51567	eval-rmse:3.66965
[80000]	train-rmse:3.50739	eval-rmse:3.66937
[85000]	train-rmse:3.4993	eval-rmse:3.66916
[90000]	train-rmse:3.49131	eval-rmse:3.66894
[9500

In [ ]:
sub_df = pd.DataFrame({"card_id":test_cardid.values})
sub_df["target"] = predictions
sub_df.to_csv("submit_with_all_additionsXGBOOST.csv", index=False)

In [80]:
best_test,best_x_test

(3.654300059447733, 90)

In [75]:
best_test,best_x_test

(3.654603973292271, 19)

In [81]:
predictions

array([-2.59572038, -0.28360958, -1.20721482, ...,  1.12821569,
       -2.7994819 ,  0.05949883])

In [41]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

In [42]:
all_cols = pd.DataFrame(best_features.groupby(['feature'])['importance'].mean()).sort_values(by='importance')
all_cols

,importance
feature,
category_combined_with_cat1_bottom_quarter_quantile_7,1.000000
category_combined_without_cat1_top_quarter_quantile_5,1.000000
category_combined_without_cat1_modeplus_1,1.000000
category_combined_without_cat1_modeplus_16,1.000000
category_combined_without_cat1_modeplus_12,1.000000
category_combined_without_cat1_modeplus_13,1.600000
category_combined_with_cat1_top_quarter_quantile_6,1.666667
category_combined_without_cat1_modeplus_11,1.666667
agged_non_city_id_modeplus_4.0,1.666667
